In [2]:
#magnetic shielding tensor has less data than scc or train because for same index
#the mst is the same
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
from sklearn.tree import DecisionTreeRegressor

In [4]:
train_datapath = r"\\icnas2.cc.ic.ac.uk\jr3418\Desktop\UROP\Predicting the molecular properties\Data\train.csv"
train = pd.read_csv(train_datapath,)

magnetic_shielding_tensors_datapath = r"\\icnas2.cc.ic.ac.uk\jr3418\Desktop\UROP\Predicting the molecular properties\Data\magnetic_shielding_tensors.csv"
mst = pd.read_csv(magnetic_shielding_tensors_datapath)

dipolemoment_datapath = r"\\icnas2.cc.ic.ac.uk\jr3418\Desktop\UROP\Predicting the molecular properties\Data\dipole_moments.csv"
dm = pd.read_csv(dipolemoment_datapath)

test_path = r"\\icnas2.cc.ic.ac.uk\jr3418\Desktop\UROP\Predicting the molecular properties\Data\test.csv"
test = pd.read_csv(test_path, )

mulliken_charge_path = r"\\icnas2.cc.ic.ac.uk\jr3418\Desktop\UROP\Predicting the molecular properties\Data\mulliken_charges.csv"
mc = pd.read_csv(mulliken_charge_path)

potentialenergy_path = r"\\icnas2.cc.ic.ac.uk\jr3418\Desktop\UROP\Predicting the molecular properties\Data\potential_energy.csv"
pe = pd.read_csv(potentialenergy_path)

scalar_coupling_contributions_path = r"\\icnas2.cc.ic.ac.uk\jr3418\Desktop\UROP\Predicting the molecular properties\Data\scalar_coupling_contributions.csv"
scc = pd.read_csv(scalar_coupling_contributions_path)

structures_path = r"\\icnas2.cc.ic.ac.uk\jr3418\Desktop\UROP\Predicting the molecular properties\Data\structures.csv"
structures = pd.read_csv(structures_path)

In [5]:
print(f'There are {train.shape[0]} rows in train data.')
print(f'There are {test.shape[0]} rows in test data.')

print(f"There are {train['molecule_name'].nunique()} distinct molecules in train data.")
print(f"There are {test['molecule_name'].nunique()} distinct molecules in test data.")
print(f"There are {structures['atom'].nunique()} unique atoms.")
print(f"There are {train['type'].nunique()} unique types.")

There are 4658147 rows in train data.
There are 2505542 rows in test data.
There are 85003 distinct molecules in train data.
There are 45772 distinct molecules in test data.
There are 5 unique atoms.
There are 8 unique types.


In [6]:
test.head(10)

,id,molecule_name,atom_index_0,atom_index_1,type
0,4658147,dsgdb9nsd_000004,2,0,2JHC
1,4658148,dsgdb9nsd_000004,2,1,1JHC
2,4658149,dsgdb9nsd_000004,2,3,3JHH
3,4658150,dsgdb9nsd_000004,3,0,1JHC
4,4658151,dsgdb9nsd_000004,3,1,2JHC
5,4658152,dsgdb9nsd_000015,3,0,1JHC
6,4658153,dsgdb9nsd_000015,3,2,3JHC
7,4658154,dsgdb9nsd_000015,3,4,2JHH
8,4658155,dsgdb9nsd_000015,3,5,2JHH
9,4658156,dsgdb9nsd_000015,4,0,1JHC


In [9]:
mst.tail(20)

,molecule_name,atom_index,XX,YX,ZX,XY,YY,ZY,XZ,YZ,ZZ
1533517,dsgdb9nsd_133882,14,28.4390,-4.4719,0.0000,-2.5712,26.2508,-0.0000,0.0000,0.0000,23.2280
1533518,dsgdb9nsd_133882,15,28.2781,-0.1637,2.5996,1.1657,25.9426,2.1164,2.2967,-0.5841,31.6971
1533519,dsgdb9nsd_133884,0,114.0670,-4.0079,-0.0000,1.6178,142.5280,-0.0002,-0.0001,-0.0002,124.8530
1533520,dsgdb9nsd_133884,1,156.2800,4.6809,0.0028,52.2123,70.1405,0.0005,0.0026,0.0006,147.8480
1533521,dsgdb9nsd_133884,2,120.2310,-6.9024,-9.9702,-18.4376,109.9950,28.7199,-1.5542,9.5272,89.9296
1533522,dsgdb9nsd_133884,3,196.1870,-21.3621,7.9866,-20.3383,153.3530,11.5939,-1.4026,11.3753,144.3600
1533523,dsgdb9nsd_133884,4,175.5090,-26.4784,0.0000,-32.8936,140.6690,0.0002,0.0012,0.0002,136.0550
1533524,dsgdb9nsd_133884,5,164.8640,2.7385,-0.0001,-0.5899,170.0420,0.0002,-0.0002,0.0003,143.3250
1533525,dsgdb9nsd_133884,6,140.8190,1.6914,-0.0002,39.5850,168.6970,-0.0006,-0.0002,-0.0006,103.1220
1533526,dsgdb9nsd_133884,7,120.2300,-6.9030,9.9705,-18.4385,109.9950,-28.7188,1.5550,-9.5259,89.9291


In [10]:
scc.tail(20)

,molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso,dso
4658127,dsgdb9nsd_133884,15,7,3JHC,6.295230,0.015189,0.227815,-0.407014
4658128,dsgdb9nsd_133884,15,8,3JHC,5.264210,-0.015521,0.304764,-0.429302
4658129,dsgdb9nsd_133884,16,0,3JHC,4.925530,-0.058898,-0.017907,0.073951
4658130,dsgdb9nsd_133884,16,1,2JHN,3.506410,0.027852,0.203386,-0.018397
4658131,dsgdb9nsd_133884,16,2,3JHC,9.241950,-0.064811,0.302867,-0.544545
4658132,dsgdb9nsd_133884,16,3,3JHC,3.726740,-0.035967,0.349909,-0.495334
4658133,dsgdb9nsd_133884,16,4,3JHC,-0.144487,-0.023683,0.201759,-0.278366
4658134,dsgdb9nsd_133884,16,5,3JHC,-0.063559,-0.033296,0.065262,-0.153096
4658135,dsgdb9nsd_133884,16,6,2JHC,0.790540,0.147738,0.131503,-0.044294
4658136,dsgdb9nsd_133884,16,7,1JHC,98.241000,0.290716,-0.151399,1.276900


In [11]:
data = pd.merge(train,mst, on = ['molecule_name'])

In [12]:
data.tail(20)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_index,XX,YX,ZX,XY,YY,ZY,XZ,YZ,ZZ
88706505,4658145,dsgdb9nsd_133884,17,7,2JHC,4.76201,16,28.9460,0.0146,2.5359,-2.0440,24.9160,-1.7686,1.8798,0.9608,31.6697
88706506,4658145,dsgdb9nsd_133884,17,7,2JHC,4.76201,17,30.1753,-6.4689,-5.5474,-3.8417,30.4761,6.3612,-1.5380,4.0582,29.4963
88706507,4658146,dsgdb9nsd_133884,17,8,1JHC,117.93400,0,114.0670,-4.0079,-0.0000,1.6178,142.5280,-0.0002,-0.0001,-0.0002,124.8530
88706508,4658146,dsgdb9nsd_133884,17,8,1JHC,117.93400,1,156.2800,4.6809,0.0028,52.2123,70.1405,0.0005,0.0026,0.0006,147.8480
88706509,4658146,dsgdb9nsd_133884,17,8,1JHC,117.93400,2,120.2310,-6.9024,-9.9702,-18.4376,109.9950,28.7199,-1.5542,9.5272,89.9296
88706510,4658146,dsgdb9nsd_133884,17,8,1JHC,117.93400,3,196.1870,-21.3621,7.9866,-20.3383,153.3530,11.5939,-1.4026,11.3753,144.3600
88706511,4658146,dsgdb9nsd_133884,17,8,1JHC,117.93400,4,175.5090,-26.4784,0.0000,-32.8936,140.6690,0.0002,0.0012,0.0002,136.0550
88706512,4658146,dsgdb9nsd_133884,17,8,1JHC,117.93400,5,164.8640,2.7385,-0.0001,-0.5899,170.0420,0.0002,-0.0002,0.0003,143.3250
88706513,4658146,dsgdb9nsd_133884,17,8,1JHC,117.93400,6,140.8190,1.6914,-0.0002,39.5850,168.6970,-0.0006,-0.0002,-0.0006,103.1220
88706514,4658146,dsgdb9nsd_133884,17,8,1JHC,117.93400,7,120.2300,-6.9030,9.9705,-18.4385,109.9950,-28.7188,1.5550,-9.5259,89.9291


In [14]:
train.tail()

,id,molecule_name,atom_index,atom_index_1,type,scalar_coupling_constant
1048570,1048570,dsgdb9nsd_036283,13,7,3JHC,2.545950
1048571,1048571,dsgdb9nsd_036283,13,8,3JHC,2.580430
1048572,1048572,dsgdb9nsd_036283,13,14,2JHH,-5.081760
1048573,1048573,dsgdb9nsd_036283,13,15,3JHH,-0.416883
1048574,1048574,dsgdb9nsd_036283,14,3,3JHC,1.208020


In [15]:
train_datapath = r"\\icnas2.cc.ic.ac.uk\jr3418\Desktop\UROP\Predicting the molecular properties\Data\train.csv"
train = pd.read_csv(train_datapath,)
train.tail()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
4658142,4658142,dsgdb9nsd_133884,17,4,2JHC,3.543450
4658143,4658143,dsgdb9nsd_133884,17,5,3JHC,0.568997
4658144,4658144,dsgdb9nsd_133884,17,6,3JHC,1.173370
4658145,4658145,dsgdb9nsd_133884,17,7,2JHC,4.762010
4658146,4658146,dsgdb9nsd_133884,17,8,1JHC,117.934000


In [17]:
train.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type',
       'scalar_coupling_constant'],
      dtype='object')

In [6]:
train.columns = (['id', 'molecule_name', 'atom_index', 'atom_index_1', 'type',
       'scalar_coupling_constant'])

In [20]:
train.head()

,id,molecule_name,atom_index,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [10]:
data = pd.merge(train,mst,on =['atom_index'])

MemoryError: 